In [17]:
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV,RandomizedSearchCV
from sklearn.feature_selection import RFECV,SelectFromModel
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
from sklearn.feature_selection import VarianceThreshold
import numpy as np
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from sklearn.model_selection import ParameterSampler
from scipy.stats import uniform, randint
import pandas as pd
from scipy.stats import randint



In [18]:
import pandas as pd
train=pd.read_csv('../data/training_data.csv',low_memory=True, dtype=np.float32)
test=pd.read_csv('../data/testing_data.csv',low_memory=True, dtype=np.float32)



In [19]:
dep = ['Label']
X_train = train.loc[:,~train.columns.isin(dep)]
y_train = train.loc[:,train.columns.isin(dep)]

In [20]:
X_val = test.loc[:,~test.columns.isin(dep)]
y_val = test.loc[:,test.columns.isin(dep)]

In [21]:
# Fit base model on all features
base_model = XGBClassifier(n_estimators=100, random_state=42, use_label_encoder=False, eval_metric='logloss')
base_model.fit(X_train, y_train)

# Select important features only
selector = SelectFromModel(base_model, threshold="median", prefit=True)
X_train_sel = selector.transform(X_train)
X_val_sel = selector.transform(X_val)

c:\Users\ASUS\Desktop\Assignment - Sigmoid\model-dev\Lib\site-packages\xgboost\training.py:183: UserWarning: [00:43:39] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\ASUS\Desktop\Assignment - Sigmoid\model-dev\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
c:\Users\ASUS\Desktop\Assignment - Sigmoid\model-dev\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [22]:
y_train.mean()

Label    0.073314
dtype: float32

In [23]:
# Define param grid
param_dist = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(2, 8),
    'learning_rate': uniform(0.01, 0.2),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'reg_alpha': uniform(0, 0.5),
    'reg_lambda': uniform(0.5, 3)
}

n_iter = 5
param_list = list(ParameterSampler(param_dist, n_iter=n_iter, random_state=42))

best_score = -1
best_model = None
best_params = None
results = []

# Loop over parameter sets manually
for i, params in enumerate(param_list):
    print(params)
    model = XGBClassifier(
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=42,
        **params
    )
    model.fit(X_train_sel, y_train)

    # Score on your external validation set
    y_val_prob = model.predict_proba(X_val_sel)[:, 1]
    val_auc = roc_auc_score(y_val, y_val_prob)
    print(val_auc)
    results.append({'params': params, 'val_auc': val_auc})

    if val_auc > best_score:
        best_score = val_auc
        best_model = model
        best_params = params

print(f"\nBest AUC on Validation: {best_score:.4f}")
print("Best Parameters:", best_params)

{'colsample_bytree': np.float64(0.749816047538945), 'learning_rate': np.float64(0.20014286128198325), 'max_depth': 4, 'n_estimators': 171, 'reg_alpha': np.float64(0.2993292420985183), 'reg_lambda': np.float64(0.9680559213273096), 'subsample': np.float64(0.662397808134481)}


c:\Users\ASUS\Desktop\Assignment - Sigmoid\model-dev\Lib\site-packages\xgboost\training.py:183: UserWarning: [00:43:55] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


0.5749724535367368
{'colsample_bytree': np.float64(0.6232334448672797), 'learning_rate': np.float64(0.18323522915498705), 'max_depth': 5, 'n_estimators': 459, 'reg_alpha': np.float64(0.35403628889802274), 'reg_lambda': np.float64(0.5617534828874073), 'subsample': np.float64(0.9879639408647978)}


c:\Users\ASUS\Desktop\Assignment - Sigmoid\model-dev\Lib\site-packages\xgboost\training.py:183: UserWarning: [00:44:10] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


0.5694464690158579
{'colsample_bytree': np.float64(0.9329770563201687), 'learning_rate': np.float64(0.052467822135655234), 'max_depth': 5, 'n_estimators': 376, 'reg_alpha': np.float64(0.30874075481385826), 'reg_lambda': np.float64(2.3349594814648427), 'subsample': np.float64(0.6028265220878869)}


...

0.5803012592631733
{'colsample_bytree': np.float64(0.6092249700165663), 'learning_rate': np.float64(0.11495493205167782), 'max_depth': 3, 'n_estimators': 287, 'reg_alpha': np.float64(0.18318092164684585), 'reg_lambda': np.float64(1.8682099526511078), 'subsample': np.float64(0.9140703845572055)}


c:\Users\ASUS\Desktop\Assignment - Sigmoid\model-dev\Lib\site-packages\xgboost\training.py:183: UserWarning: [00:45:38] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


0.5812754071058812
{'colsample_bytree': np.float64(0.6798695128633439), 'learning_rate': np.float64(0.11284688768272232), 'max_depth': 2, 'n_estimators': 230, 'reg_alpha': np.float64(0.4299702033681603), 'reg_lambda': np.float64(2.540922615763339), 'subsample': np.float64(0.7801997007878172)}


c:\Users\ASUS\Desktop\Assignment - Sigmoid\model-dev\Lib\site-packages\xgboost\training.py:183: UserWarning: [00:45:56] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


0.5852404579129347

Best AUC on Validation: 0.5852
Best Parameters: {'colsample_bytree': np.float64(0.6798695128633439), 'learning_rate': np.float64(0.11284688768272232), 'max_depth': 2, 'n_estimators': 230, 'reg_alpha': np.float64(0.4299702033681603), 'reg_lambda': np.float64(2.540922615763339), 'subsample': np.float64(0.7801997007878172)}


In [24]:
results

[{'params': {'colsample_bytree': np.float64(0.749816047538945),
   'learning_rate': np.float64(0.20014286128198325),
   'max_depth': 4,
   'n_estimators': 171,
   'reg_alpha': np.float64(0.2993292420985183),
   'reg_lambda': np.float64(0.9680559213273096),
   'subsample': np.float64(0.662397808134481)},
  'val_auc': np.float64(0.5749724535367368)},
 {'params': {'colsample_bytree': np.float64(0.6232334448672797),
   'learning_rate': np.float64(0.18323522915498705),
   'max_depth': 5,
   'n_estimators': 459,
   'reg_alpha': np.float64(0.35403628889802274),
   'reg_lambda': np.float64(0.5617534828874073),
   'subsample': np.float64(0.9879639408647978)},
  'val_auc': np.float64(0.5694464690158579)},
 {'params': {'colsample_bytree': np.float64(0.9329770563201687),
   'learning_rate': np.float64(0.052467822135655234),
   'max_depth': 5,
   'n_estimators': 376,
   'reg_alpha': np.float64(0.30874075481385826),
   'reg_lambda': np.float64(2.3349594814648427),
   'subsample': np.float64(0.602826

In [25]:
best_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=np.float64(0.6798695128633439), device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='logloss', feature_types=None, feature_weights=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None,
              learning_rate=np.float64(0.11284688768272232), max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=230, n_jobs=None,
              num_parallel_tree=None, ...)

In [26]:
best_model.save_model("../model-objects/xgb_final_model.json")  # Saves model to JSON format
# #loaded_model = XGBClassifier()
# #loaded_model.load_model("../model-objects/xgb_final_model.json")


In [27]:
y_train_prob = best_model.predict_proba(X_train_sel)[:, 1]
y_val_prob = best_model.predict_proba(X_val_sel)[:, 1]

In [28]:
data_ap_train = pd.DataFrame({'Actual':y_train.values.ravel(),'Predicted':y_train_prob})
data_ap_val = pd.DataFrame({'Actual':y_val.values.ravel(),'Predicted':y_val_prob})

**KS AND RANK ORDER**

In [29]:
def ks(data=None,target=None, prob=None):
    data['target0'] = 1 - data[target]
    data['bucket'] = pd.qcut(data[prob], 10)
    grouped = data.groupby('bucket', as_index = False)
    kstable = pd.DataFrame()
    kstable['min_prob'] = grouped.min()[prob]
    kstable['max_prob'] = grouped.max()[prob]
    kstable['events']   = grouped.sum()[target]
    kstable['nonevents'] = grouped.sum()['target0']
    kstable = kstable.sort_values(by="min_prob", ascending=False).reset_index(drop = True)
    kstable['event_rate'] = (kstable.events / data[target].sum()).apply('{0:.2%}'.format)
    kstable['nonevent_rate'] = (kstable.nonevents / data['target0'].sum()).apply('{0:.2%}'.format)
    kstable['cum_eventrate']=(kstable.events / data[target].sum()).cumsum()
    kstable['cum_noneventrate']=(kstable.nonevents / data['target0'].sum()).cumsum()
    kstable['KS'] = np.round(kstable['cum_eventrate']-kstable['cum_noneventrate'], 3) * 100

    #Formating
    kstable['cum_eventrate']= kstable['cum_eventrate'].apply('{0:.2%}'.format)
    kstable['cum_noneventrate']= kstable['cum_noneventrate'].apply('{0:.2%}'.format)
    kstable.index = range(1,11)
    kstable.index.rename('Decile', inplace=True)
    pd.set_option('display.max_columns', 9)
    print(kstable)
    
    #Display KS
    from colorama import Fore
    print(Fore.RED + "KS is " + str(max(kstable['KS']))+"%"+ " at decile " + str((kstable.index[kstable['KS']==max(kstable['KS'])][0])))
    return(kstable)

In [30]:
ks_train=ks(data=data_ap_train,target='Actual',prob='Predicted')

        min_prob  max_prob  events  nonevents event_rate nonevent_rate  \
Decile                                                                   
1       0.208443  0.997707  2028.0     1472.0     79.03%         4.54%   
2       0.064242  0.208398   374.0     3126.0     14.58%         9.64%   
3       0.027749  0.064232   104.0     3396.0      4.05%        10.47%   
4       0.013676  0.027745    37.0     3463.0      1.44%        10.68%   
5       0.007272  0.013674    17.0     3483.0      0.66%        10.74%   
6       0.004067  0.007269     4.0     3496.0      0.16%        10.78%   
7       0.002328  0.004067     1.0     3499.0      0.04%        10.79%   
8       0.001284  0.002328     1.0     3499.0      0.04%        10.79%   
9       0.000640  0.001284     0.0     3500.0      0.00%        10.79%   
10      0.000018  0.000640     0.0     3500.0      0.00%        10.79%   

       cum_eventrate cum_noneventrate         KS  
Decile                                            
1        

C:\Users\ASUS\AppData\Local\Temp\ipykernel_9308\572846000.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = data.groupby('bucket', as_index = False)


In [31]:
ks_val=ks(data=data_ap_val,target='Actual',prob='Predicted')

        min_prob  max_prob  events  nonevents event_rate nonevent_rate  \
Decile                                                                   
1       0.149958  0.997131   180.0     1320.0     16.27%         9.50%   
2       0.055485  0.149937   151.0     1349.0     13.65%         9.71%   
3       0.025536  0.055420   124.0     1376.0     11.21%         9.90%   
4       0.013305  0.025528   112.0     1388.0     10.13%         9.99%   
5       0.007407  0.013280   118.0     1382.0     10.67%         9.95%   
6       0.004038  0.007396    96.0     1404.0      8.68%        10.11%   
7       0.002292  0.004035    85.0     1415.0      7.69%        10.18%   
8       0.001233  0.002290    89.0     1411.0      8.05%        10.16%   
9       0.000555  0.001232    79.0     1421.0      7.14%        10.23%   
10      0.000030  0.000555    72.0     1428.0      6.51%        10.28%   

       cum_eventrate cum_noneventrate    KS  
Decile                                       
1             16.27

C:\Users\ASUS\AppData\Local\Temp\ipykernel_9308\572846000.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = data.groupby('bucket', as_index = False)


In [32]:
ks_train.to_csv('../data-analysis/ks_table_train_xgb.csv')
ks_val.to_csv('../data-analysis/ks_table_val_xgb.csv')